# Fuel Moisture Scarios for Testing

*Jonathon Hirschi*


The purpose of this notebook is to generate fuel moisture scenarios for  model testing.

In [ ]:
# both can change
# Environment
import numpy as np, random
import pandas as pd
from MesoPy import Meso
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_raws, retrieve_raws, format_precip, fixnan, create_synthetic_data, model_decay, synthetic_data

meso_token="4192c18707b848299783d59a9317c6e1"
m=Meso(meso_token)

In [ ]:

def synthetic_data(days=20,power=4,data_noise=0.02,process_noise=0.0,DeltaE=0.0,Emin=0.5,Emax=0.3, scale = False):
    hours = days*24
    h2 = int(hours/2)
    hour = np.array(range(hours))
    day = np.array(range(hours))/24.
    # artificial equilibrium data
    E = np.power(np.sin(np.pi*day),power) # diurnal curve
    E = Emin+(Emax - Emin)*E
    # FMC free run
    m_f = np.zeros(hours)
    m_f[0] = 0.1         # initial FMC
    # process_noise=0.
    for t in range(hours-1):
        m_f[t+1] = max(0.,model_decay(m_f[t],E[t])  + random.gauss(0,process_noise) )
    data = m_f + np.random.normal(loc=0,scale=data_noise,size=hours)
    E = E + DeltaE    
    Ed=E+1.0
    Ew=np.maximum(E-1.0,0)
    
    if scale: E=E*100; Ew=Ew*100; Ed=Ed*100; m_f=m_f*100; data=data*100
    
    return {'E':E,'Ew':Ew,'Ed':Ed,'m_f':m_f, 'data':data,'hour':hour,'h2':h2,'DeltaE':DeltaE}

In [ ]:
random.seed(123)
np.random.seed(123)

In [ ]:
dict1 = synthetic_data(days=20,power=4,data_noise=0.01,process_noise=0.0,DeltaE=0.0, Emin=.6, Emax=.8) 

In [ ]:
dict1['rain'] = np.zeros(len(dict1['hour']))

In [ ]:
def plot_data(fm_dict):
    h2 = fm_dict['h2']
    plt.figure(figsize=(16,4))
    plt.plot(fm_dict['hour'],fm_dict['E'],linestyle='-',c='r',label='Equilibrium', alpha=.8)
    plt.plot(fm_dict['Ed'],linestyle=':',c='r',label='drying equilibrium', alpha=.5)
    plt.plot(fm_dict['Ew'],linestyle=':',c='b',label='wetting equilibrium', alpha=.5)
    plt.plot(fm_dict['hour'],fm_dict['m_f'],linestyle='-',c='k',label='10-h fuel truth')
    plt.scatter(fm_dict['hour'][:h2],fm_dict['data'][:h2],c='k',label='10-h fuel data', alpha=.5, s = 15)
    plt.title('Synthetic data')
    plt.xlabel('Time (hours)')
    plt.ylabel('Fuel moisture content (%)')
    plt.plot(fm_dict['hour'],fm_dict['rain'],linestyle='-',c='b',label='Rain (mm/h)', alpha=.5)
    plt.legend()

In [ ]:
plot_data(dict1)

[USGS Rainfall Guide](https://water.usgs.gov/edu/activity-howmuchrain-metric.html#:~:text=Slight%20rain%3A%20Less%20than%200.5,than%208%20mm%20per%20hour.)

In [ ]:
# Step Function Rain
first_rain = int(hours/2) # hours of first rain
rain0 = np.zeros(len(dict1['hour']))
rain1 = .25*np.heaviside(hour - first_rain, 1) # slight rain, .25 mm/hr
rain2 = 2*np.heaviside(hour - first_rain, 1) # moderate rain, 2 mm/hr
rain3 = 5*np.heaviside(hour - first_rain, 1) # heavy rain, 5 mm/hr
rain4 = 10*np.heaviside(hour - first_rain, 1) # very heavy rain, 10 mm/hr

In [ ]:
days = 5
hours = days*24
h2 = int(hours/2)
hour = np.array(range(hours))
day = np.array(range(hours))/24.


rh = 5*np.sin(4*np.pi*day)+20
print(rh.mean())
print(rh.var())
print(rh.max())
print(rh.min())
plt.plot(hour, rh)
plt.plot(hour[:24], rh[:24])

In [ ]:
# compute relative humidity
t2 = np.array(rtma['temp']) # surface temp
rh = 100*np.exp(17.625*243.04*(td - t2) / (243.04 + t2 - 273.15) / (243.0 + td - 273.15))
Ed = 0.924*rh**0.679 + 0.000499*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - t2)*(1 - np.exp(-0.115*rh))
Ew = 0.618*rh**0.753 + 0.000454*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - t2)*(1 - np.exp(-0.115*rh)))

In [ ]:
E,m_f,data,hour,h2,DeltaE = create_synthetic_data(days=20,power=4,data_noise=0.01,process_noise=0.0,DeltaE=0.1) 